# Binary Image Classifier

In [1]:
# import libraries and dependencies
import tensorflow as tf 
import tensorflow_hub as hub 
import os
import numpy as np 
import shutil

In [2]:
# replace with your own directory location

test_dir = 'F:/Nilgai_photo_database/Nilgai Classifier/Nilgai/sample_images/'
img_dir = 'F:/Nilgai_photo_database/Nilgai Classifier/Nilgai/sample_images/images/'
pred_dir = 'F:/Nilgai_photo_database/Nilgai Classifier/Nilgai/predictions/'
binary_model_path = 'F:/Nilgai_photo_database/Nilgai Classifier/Nilgai/models/Final_binary_nil_05262020.h5'

In [3]:
BATCH_SIZE = 1
IMG_HEIGHT = 299
IMG_WIDTH = 299

def img_crop(img):
    # for cropping
    offset_width = 0
    offset_height = 9
    target_height = 273
    target_width = 299

    new_img = tf.image.crop_to_bounding_box(img, offset_height, offset_width, target_height, target_width)
    new_img = tf.image.resize(new_img, (299,299))
    return new_img

In [4]:
# tensorflow data loaders
img_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, preprocessing_function=img_crop)

test_generator = img_generator.flow_from_directory(directory=str(test_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=False,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode= None)
# load model
binary_model = tf.keras.models.load_model(binary_model_path, custom_objects={'KerasLayer':hub.KerasLayer}, compile=True)

Found 100 images belonging to 1 classes.


In [5]:
# make predictions
pred_steps=np.ceil(test_generator.samples/test_generator.batch_size)

pred = binary_model.predict(test_generator, verbose=1, workers=4, steps=pred_steps)

100/100 [==============================] - 2s 17ms/step


In [6]:
# define class names and create and index for future mapping
predicted_class_indices = tf.round(pred).numpy().flatten()
class_names = {'Nilgai': 0, 'Not_Nilgai': 1}
labels = dict((v,k) for k,v in class_names.items())
predictions = [labels[k] for k in predicted_class_indices]

# predicted_id = np.round(pred).astype(int).flatten()
# predicted_label_batch = class_names[predicted_id]
# predicted_label_batch
predictions[3]


'Not_Nilgai'

In [7]:
for i in class_names:
    img_path = os.path.join(pred_dir, str(i))
    try:
        os.makedirs(img_path, exist_ok=False)
    except:
        print("The " + '"' + str(i) + '"' + " folder already exists. Check your prediction path.")

The "Nilgai" folder already exists. Check your prediction path.
The "Not_Nilgai" folder already exists. Check your prediction path.


In [38]:
## Map prediction name to labels, move images to repsective folders, and print results
## Todo: make this a function
for i, logits in enumerate(pred):

    conf = pd.DataFrame({'Prediction':list(pred)})
    conf = conf.Prediction.apply(pd.Series)
    conf = conf.max(axis=1)


    class_idx = tf.round(pred).numpy().flatten()
    labels = {'Nilgai': 0, 'Not_Nilgai': 1}
    labels = dict((v,k) for k,v in labels.items())
    predictions = [labels[k] for k in predicted_class_indices]
    predictions = predictions[i]

    img_name = os.path.split(test_generator.filenames[i])[1]
    samp_name = os.path.splitext(img_name)[0]
    img = os.path.join(img_dir,img_name)
    dst = os.path.join(pred_dir, predictions)
    shutil.copy(img, dst)
    
    # print("{} prediction: {} ({:4.2f}%)".format(str(samp_name), str(name), 100*pred[i].item()))
    print(labels)
    print(samp_name)
    print(img_name)
    print(i)
    print(predictions)
    print()

{0: 'Nilgai', 1: 'Not_Nilgai'}
0
0.jpg
0
Not_Nilgai

{0: 'Nilgai', 1: 'Not_Nilgai'}
1
1.jpg
1
Not_Nilgai

{0: 'Nilgai', 1: 'Not_Nilgai'}
10
10.jpg
2
Not_Nilgai

{0: 'Nilgai', 1: 'Not_Nilgai'}
11
11.jpg
3
Not_Nilgai

{0: 'Nilgai', 1: 'Not_Nilgai'}
12
12.jpg
4
Not_Nilgai

{0: 'Nilgai', 1: 'Not_Nilgai'}
13
13.jpg
5
Not_Nilgai

{0: 'Nilgai', 1: 'Not_Nilgai'}
14
14.jpg
6
Not_Nilgai

{0: 'Nilgai', 1: 'Not_Nilgai'}
15
15.jpg
7
Not_Nilgai

{0: 'Nilgai', 1: 'Not_Nilgai'}
16
16.jpg
8
Not_Nilgai

{0: 'Nilgai', 1: 'Not_Nilgai'}
17
17.jpg
9
Not_Nilgai

{0: 'Nilgai', 1: 'Not_Nilgai'}
18
18.jpg
10
Not_Nilgai

{0: 'Nilgai', 1: 'Not_Nilgai'}
19
19.jpg
11
Nilgai

{0: 'Nilgai', 1: 'Not_Nilgai'}
2
2.jpg
12
Not_Nilgai

{0: 'Nilgai', 1: 'Not_Nilgai'}
20
20.jpg
13
Not_Nilgai

{0: 'Nilgai', 1: 'Not_Nilgai'}
21
21.jpg
14
Not_Nilgai

{0: 'Nilgai', 1: 'Not_Nilgai'}
22
22.jpg
15
Not_Nilgai

{0: 'Nilgai', 1: 'Not_Nilgai'}
23
23.jpg
16
Not_Nilgai

{0: 'Nilgai', 1: 'Not_Nilgai'}
24
24.jpg
17
Not_Nilgai

{0: 'Nilgai'

In [11]:
print("hello")

hello


In [25]:
import pandas as pd


In [28]:
conf[1]

0.9999659